## Work very much in progress

In [1]:
import numpy as np
import scipy as sp
import scipy.spatial.distance as dist
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import random
import pandas as pd
# Custom libraries
import CCM_GAH_lib as ccm
import data_handle_lib as dh

#### Two-species model system with synchrony

In [2]:
# Generate artificial data 

def f_x (f_x_prev, f_y_prev):
    return f_x_prev*(3.8 - 3.8*f_x_prev) # - 0.07*f_y_prev)

def f_y (f_x_prev, f_y_prev):
    return f_y_prev*(3.1 - 3.1*f_y_prev - 0.8*f_x_prev)

time_max = 4000
t = np.arange(time_max)
x = np.zeros(time_max)
x[0] = 0.2 # from supplementary materials
y = np.zeros(time_max)
y[0] = 0.4 # from supplementary materials

for i in range(time_max-1):
    x[i+1] = f_x(x[i], y[i])
    y[i+1] = f_y(x[i], y[i])

In [3]:
#plt.plot(t[0:100], x[0:100])
#plt.plot(t[0:100], y[0:100])
#plt.show()

In [4]:
E = 2
tau = 1
how_many = 2
how_long = 200

lag_min, lag_max = -8, 8
lag_array = np.arange(lag_min, lag_max)

rho_xcrossy, rho_ycrossx = pd.DataFrame(columns=lag_array), pd.DataFrame(columns=lag_array)

In [ ]:
for i in range(how_many):
    rand_start = random.randint(1, 2000-how_long)
    x_orig = np.concatenate((x[rand_start:rand_start+how_long], x[2000:3000]))
    y_orig = np.concatenate((y[rand_start:rand_start+how_long], y[2000:3000]))
    
    y_approx = ccm.generateYApprox(x_orig, y_orig, E=E, tau=tau, leaveOut=False)
    x_approx = ccm.generateYApprox(y_orig, x_orig, E=E, tau=tau, leaveOut=False)
    
    xcrossy, ycrossx = [], []
    for lag in lag_array:
        start = (E-1)+10
        end = len(x_orig)-10
        xcrossy.append(pearsonr(x_orig[start+lag:end+lag], y_approx[start:end])[0])
        ycrossx.append(pearsonr(y_orig[start+lag:end+lag], y_approx[start:end])[0])
    
    rho_xcrossy.loc[i] = xcrossy
    rho_ycrossx.loc[i] = ycrossx

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(rho_xcrossy.loc[i])
plt.plot(rho_ycrossx.loc[i])
plt.legend(["x cross y", "y cross x"])
plt.show()